In [ ]:
#@title setup, static and constant data

# Mount Google Drive
import os
from google.colab import drive
gdrive = '/content/drive/'
gdriveprefix = gdrive + 'My Drive/Colab Notebooks/'
drive.mount(gdrive, force_remount=True)

import numpy as np
import pandas as pd
import urllib.request

# useful constants
tab = "\t";
newline = "\n";

# sitelist data locations
data_prefix ='https://raw.githubusercontent.com/bdekoz/mozilla-data-lcp/main/';
sitebase = 'CrUX_global_200_10M-2023-12';
sitelist = sitebase + '.txt.ping';
sitefile = data_prefix + "sitelists/" + sitelist;

# sitelist currently reachable
errfile = gdriveprefix + sitebase + ".error.txt";
okfile = gdriveprefix + sitebase + ".pass.txt";




Mounted at /content/drive/


In [ ]:
#@title percentage(numerator, denom)

def percentage(part, whole):
  Percentage = 100 * float(part)/float(whole)
  return str(int(round(Percentage,0))) + '%'




In [ ]:
#@title origin_check_readable(origin, log) / origin_contains(origin, match)

import requests

# check origin to see if it can be read
def origin_check_readable(origin, logfile):
  try:
    r = requests.get(origin);
  except:
    logfile.write(origin + newline);


def origin_contains(origin, match, logfile):
  matchp = False
  r = requests.get(origin);
  if match in r.text:
    matchp = True
  return matchp

In [ ]:
#@title check sitelist

with urllib.request.urlopen(sitefile) as response:
  print("found: " + sitefile);

  errlog = open(errfile, "w");
  passlog = open(okfile, "w");
  print(errfile);
  print(okfile);

  for line in response.readlines():
    origin = line.decode("ascii").strip(newline); # utf-8, ascii
    try:
      origin_check_readable(origin, errlog);
      passlog.write(origin + newline);
    except:
      continue;

  errlog.close();
  passlog.close();

found: https://raw.githubusercontent.com/bdekoz/mozilla-data-lcp/main/sitelists/CrUX_global_200_10M-2023-12.txt.ping
/content/drive/My Drive/Colab Notebooks/CrUX_global_200_10M-2023-12.error.txt
/content/drive/My Drive/Colab Notebooks/CrUX_global_200_10M-2023-12.pass.txt


In [ ]:
#@title create_content_setasides(sitefile, tag, match)

# Open the URL and read the contents, establish sitelist size.
siten=0;
with urllib.request.urlopen(sitefile) as response:
  siten = len(response.readlines())
print("sites: " + str(siten))


# Web Content string fragments
# <link rel="preconnect"
# <link rel="dns-prefetch"
# https://developer.mozilla.org/en-US/docs/Web/Performance/Speculative_loading
# https://developer.mozilla.org/en-US/docs/Web/Performance/dns-prefetch
wc1='rel="dns-prefetch"';
wc2='rel="preconnect"';
wc3='rel="preload"';
wc4='rel="prefetch"';
wc5='rel="prerender"';

def create_content_traits(sitefile, tag, matchstr):
  log = open(gdriveprefix + tag + ".error.log", "w");
  df = pd.DataFrame(columns=[tag], index=range(siten))

  # Scan and store data for lookups.
  with urllib.request.urlopen(sitefile) as response:
    linen = 0
    for line in response.readlines():
      origin = line.decode("ascii").strip(newline); # utf-8, ascii

      try:
        matchp = False
        r = requests.get(origin);
        if matchstr in r.text:
          matchp = True
        print(str(linen) + tab + str(matchp));
        df.iloc[linen, 0] = matchp;
      except:
        log.write(str(linen) + ',' + origin + newline);
        df.iloc[linen, 0] = np.NaN;
        continue;
      finally:
        linen += 1;

  print("finished");
  log.close();
  df.to_csv(gdriveprefix + tag + ".csv");

#create_content_traits(sitefile, "wc1", wc1);
create_content_traits(sitefile, "wc2", wc2);
#create_content_traits(sitefile, "wc3", wc3);
#create_content_traits(sitefile, "wc4", wc4);
#create_content_traits(sitefile, "wc5", wc5);
